# Striatum example network

In this notebook we create and simulate an example striatum network. Since it is run locally we have reduced the number of neurons, so to run the full network you have to increase the neuron numbers listed below.

### Start ipcluster

Please first start ```ipcluster```, on linux you type the following into the terminal, note that you should be in the ```Snudda/examples/notebook``` directory:

```
export IPYTHONDIR="`pwd`/.ipython"
export IPYTHON_PROFILE=default
ipcluster start 
```

These commands are in ```start_ipcluster.sh``` in the ```examples/notebook``` directory. After starting ipcluster give it a few seconds to get fully up and running before proceeding with the rest of the commands in this notebook. 

If you restart the run you might need to do:

```
ipcluster stop
ipcluster start
```

For instructions how to install and activate ipyparallel for your notebook see 
https://github.com/ipython/ipyparallel

In [1]:
import os
from ipyparallel import Client

os.environ["IPYTHONDIR"] = ".ipython"
os.environ["IPYTHON_PROFILE"] = "default"

u_file = os.path.join(".ipython", "profile_default", "security", "ipcontroller-client.json")
rc = Client(url_file=u_file, timeout=120, debug=False)
d_view = rc.direct_view(targets='all')  # rc[:] # Direct view into clients

In ```struct_def``` we define the number of neurons to simulate in the striatum. Since this is an example we do a quite small network.

In [2]:
import os

network_path = os.path.join("networks/striatum_example")

from snudda.init import SnuddaInit
struct_def = {"Striatum": 500 }

si = SnuddaInit(network_path=network_path, struct_def=struct_def, random_seed=123)

Adding Striatum with 500 neurons
Using cube for striatum
Adding neurons: FSN from dir $DATA/neurons/striatum/fs
Adding neurons: dSPN from dir $DATA/neurons/striatum/dspn
Adding neurons: iSPN from dir $DATA/neurons/striatum/ispn
Adding neurons: ChIN from dir $DATA/neurons/striatum/chin
Checking boundaries, to make sure P is not too high
LTS axon density P([-0.0002],[-0.0001],[-3.e-05]) = [1.84256218e-10]
LTS axon density P([-0.0002],[-0.0001],[3.e-05]) = [1.84256218e-10]
LTS axon density P([-0.0002],[0.0001],[-3.e-05]) = [1.84256218e-10]
LTS axon density P([-0.0002],[0.0001],[3.e-05]) = [1.84256218e-10]
LTS axon density P([0.0009],[-0.0001],[-3.e-05]) = [1.72360142e-23]
LTS axon density P([0.0009],[-0.0001],[3.e-05]) = [1.72360142e-23]
LTS axon density P([0.0009],[0.0001],[-3.e-05]) = [1.72360142e-23]
LTS axon density P([0.0009],[0.0001],[3.e-05]) = [1.72360142e-23]
Adding neurons: LTS from dir $DATA/neurons/striatum/lts
Writing networks/striatum_example/network-config.json


In [3]:
from snudda.place import SnuddaPlace
sp = SnuddaPlace(network_path=network_path, rc=rc)
sp.parse_config()
sp.write_data()

In [4]:
from snudda.detect import SnuddaDetect

sd = SnuddaDetect(network_path=network_path, rc=rc)
sd.detect()

importing SnuddaDetect from snudda.detect on engine(s)
Suppressing printouts for hyper voxels that complete in < 100 seconds.
HyperID 31 completed - 5845 synapses found (104.2 s)
HyperID 41 completed - 4644 synapses found (109.3 s)
HyperID 45 completed - 7825 synapses found (126.7 s)
HyperID 21 completed - 8171 synapses found (134.6 s)
HyperID 25 completed - 18810 synapses found (141.3 s)
HyperID 46 completed - 586328 synapses found (187.8 s)
HyperID 26 completed - 1248439 synapses found (236.2 s)


In [5]:
from snudda.prune import SnuddaPrune

sp = SnuddaPrune(network_path=network_path, rc=rc)
sp.prune()

importing SnuddaPrune from snudda.prune on engine(s)
There are few synapses, we will run it in serial instead
Synapses before pruning: 1886981
Synapses after pruning: 109542 (5.81 % kept)
Gap junctions before pruning 48
Gap junctions after pruning 27 (56.25 % kept)


To generate input and simulate this network in NEURON please see ```striatum_example_simulate.ipynb```.
